# Basic Statistics and Null Model Comparasion

In [9]:
import pandas as pd

import io
import networkx as nx
import numpy as np



import matplotlib.pyplot as plt
import matplotlib as mpl


### Degree Distribution slightly editied from version given by Dr. Towlson

In [11]:
%matplotlib inline
import math
# Function Defintions
def plot_degree_dist(G):
    
    degrees = [G.degree(n) for n in G.nodes()]
    kmin = min(degrees)
    kmax = max(degrees)
    print(kmax)
    print(kmin)
    mybins = math.ceil(math.sqrt(G.number_of_nodes()))
    if kmin>0:
        bin_edges = np.logspace(np.log10(kmin), np.log10(kmax)+1, num=mybins)
    else:
        bin_edges = np.logspace(0, np.log10(kmax)+1, mybins=20)
    density, _ = np.histogram(degrees, bins=bin_edges, density=True)

    fig = plt.figure(figsize=(6,4))

    log_be = np.log10(bin_edges)
    x = 10**((log_be[1:] + log_be[:-1])/2)
    plt.loglog(x, density, marker='o', linestyle='none')
    plt.xlabel(r"degree $k$", fontsize=16)
    plt.ylabel(r"$P(k)$", fontsize=16)

    ax = plt.gca()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    sum = 0
    for i in degrees:
        sum += i
    mean = sum / len(degrees)
    print(mean)

#### Read in edges dataframe

In [13]:
edgesDf = pd.read_csv("edgesDataComp3.csv")

c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Create graph

In [17]:
edgesDf.rename(columns={'Source' : "source", 'Target' : 'target'}, inplace=True)
G = nx.from_pandas_edgelist(edgesDf, edge_attr=['Season', 'Name1', 'Name2'])

Calculate Basic statistics for Network

In [ ]:
#!pip install community
import networkx as nx
import networkx.algorithms.community as nxac
import community as community_louvain

print(G.number_of_nodes())
print(G.number_of_edges())
print(community_louvain.best_partition(G))
# print(nx.average_shortest_path_length(G,weight=None))
# print(np.mean(list(nx.clustering(G).values())))

plot_degree_dist(G)




Create 100 network null ensemble

In [4]:
def ensemble(G):
    graphList = []
    for i in range(100): # 1000 is better
        print(i)
        DP = G.copy() # the function edits the graph in place
        nx.double_edge_swap(DP,nswap=10*G.number_of_edges(), max_tries=10000000)
        curGraph = nx.to_pandas_edgelist(DP)
        curGraph.to_csv(f'random_{i}.csv', index=False)
        graphList.append(DP)
    return graphList

In [ ]:
GList = ensemble(G)

Functions for calculating communities, shortest path, and clustering coeffficent

In [5]:
def ensembleCommunities():
    #clustering_DP = []
    NumClusters = []


    for i in range(100): # 1000 is better
        randEdges = pd.read_csv(f'random_{i}.csv')
        newG = nx.from_pandas_edgelist(randEdges)
        partition = community_louvain.best_partition(newG)
        newComs = pd.DataFrame(partition.items(), columns=['ID', 'Community'])
        uniqueclusters = newComs['Community'].nunique()

        NumClusters.append(uniqueclusters)
        print(i, uniqueclusters)
            #myfile.write("\n", d_DP)
        #clustering_DP.append(c_DP)

    
    return NumClusters

In [2]:
def ensembleStatsShortestPath():
    #clustering_DP = []
    short_path_DP = []
    for i in range(100): # 1000 is better
        print(i)
        randEdges = pd.read_csv(f'random_{i}.csv')
        newG = nx.from_pandas_edgelist(randEdges)
      
        #c_DP = np.mean(list(nx.clustering(G).values()))
        if nx.is_connected(newG)==True:
            d_DP = nx.average_shortest_path_length(newG,weight=None)
            short_path_DP.append(d_DP)
        #clustering_DP.append(c_DP)

    return short_path_DP

In [ ]:
def ensembleStatsClustering():
    clustering_DP = []
    # short_path_DP = []
    for i in range(100): # 1000 is better
        print(i)
        randEdges = pd.read_csv(f'random_{i}.csv')
        newG = nx.from_pandas_edgelist(randEdges)
        c_DP = np.mean(list(nx.clustering(newG).values()))  
        clustering_DP.append(c_DP)

    return clustering_DP

In [ ]:
clusterlist = ensembleStatsClustering()
communities = ensembleCommunities()
shortpath = ensembleStatsShortestPath()

Print out results from methods

In [ ]:
print(np.mean(clusterlist))
print(np.std(clusterlist))
print(np.mean(communities))
print(np.std(communities))
print(np.mean(shortpath))
print(np.std(shortpath))
